In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

### preprocess the data

In [5]:
credit = pd.read_csv('./datasets/tmdb_5000_credits.csv')
movie = pd.read_csv('./datasets/tmdb_5000_movies.csv')

In [6]:
movie = movie.merge(credit,on='title') #merge two datasets into one

In [7]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
movie = movie[['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]

In [9]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
movie.isnull().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [11]:
movie.dropna(inplace=True) #remove null valued collumn

In [12]:
movie.isnull().sum() #no null values yay!

id          0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

In [13]:
print(movie['genres'][0]) # we have to extract genres from this
type(movie['genres'][0]) 
#we have to convert to list

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


str

In [14]:
import ast #this is module to convert str to list

def extract(d):
    ans = []
    for i in ast.literal_eval(d):
        ans.append(i['name'])
    return ans

movie['genres'] = movie['genres'].apply(extract)

In [15]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [16]:
movie['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [17]:
movie['keywords'] = movie['keywords'].apply(extract)

In [18]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [19]:
# movie['cast'][0]

In [20]:
def extract2(d):
    ans = []
    cnt = 0
    # we take top 3 character of the movie
    
    for i in ast.literal_eval(d):
        if cnt != 3:
            ans.append(i['name'])
        cnt+=1
    return ans

movie['cast'] = movie['cast'].apply(extract2)
movie['cast'] = movie['cast'].apply(lambda x:x[0:3])

In [21]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [22]:
# movie['crew'][0]

In [23]:
def extract3(d):
    ans = []
    for i in ast.literal_eval(d):
        if i['job'] == 'Director':
            ans.append(i['name'])
    return ans

movie['crew'] = movie['crew'].apply(extract3)

In [24]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [25]:
# removing space from the name avoiding duplicate name
def RemoveSpace(d):
    ans = []
    for i in d:
        ans.append(i.replace(" ",""))
    return ans

In [26]:
movie['cast'] = movie['cast'].apply(RemoveSpace)
movie['crew'] = movie['crew'].apply(RemoveSpace)
movie['genres'] = movie['genres'].apply(RemoveSpace)
movie['keywords'] = movie['keywords'].apply(RemoveSpace)

In [27]:
movie.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","In the 22nd century, a paraplegic Marine is di...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [28]:
movie['overview'] = movie['overview'].apply(lambda x: x.split())

In [29]:
movie['tags'] = movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [30]:
new = movie.drop(columns=['overview','genres','keywords','cast','crew'])

In [31]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()
# finally... yayyy!!!

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [32]:
new['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [33]:
new['tags'] = new['tags'].apply(lambda x: x.lower())

In [34]:
new['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [35]:
ps = PorterStemmer()

#this function remove same words like lover loving loved and convert to love

def stem(d):
    ans = []
    for i in d.split():
        ans.append(ps.stem(i))
        
    return ' '.join(ans)


In [36]:
new['tags'] = new['tags'].apply(stem)

In [37]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()
#vectorize the tags collumn and remove the stop words of english like 'and', 'is', 'the' etc

In [38]:
vector.shape

(4806, 5000)

In [39]:
vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [40]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [41]:
sim = cosine_similarity(vector)
#here i calculated the distance between every movies 
# here i used cos distance method to find nearest movies 

In [42]:
sim

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [44]:
sim[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [ ]:
new[new['title'] == 'Batman Begins'].index[0] #this gives us id

np.int64(119)

In [ ]:
# sorted(list(enumerate(sim[0])),reverse=True, key=lambda x:x[1])[1:11]

[(1214, np.float64(0.28676966733820225)),
 (2405, np.float64(0.26901379342448517)),
 (3728, np.float64(0.2605130246476754)),
 (507, np.float64(0.255608593705383)),
 (539, np.float64(0.25038669783359574)),
 (582, np.float64(0.24511108480187255)),
 (1202, np.float64(0.24455799402225925)),
 (1192, np.float64(0.23179316248638276)),
 (778, np.float64(0.23174488732966073)),
 (4046, np.float64(0.2278389747471728))]

In [60]:
def recommend(movie):
    idx = new[new['title'] == movie].index[0]
    dist = sorted(list(enumerate(sim[idx])),reverse=True, key=lambda x:x[1])[1:11]
    
    for i in dist:
        print(new.iloc[i[0]].title)

In [66]:
recommend('The Avengers')

Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
Captain America: The First Avenger
Iron Man
Ant-Man
Guardians of the Galaxy
X-Men
Captain America: The Winter Soldier
Fantastic 4: Rise of the Silver Surfer
